In [ ]:
%run utils.py
from functools import reduce

In [ ]:
test = """\
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

#regex_input = "(\w+) (\w+) bags contain (\d+) (\w+) (\w+) ([bag?s?,?.]+) (\d+) (\w+) (\w+) ([bag?s?,?.]+)"
#regex_input = "(\w+) (\w+) bags contain (\d+) (\w+) (\w+) ([bag?s?,?.]+)"
#regex_input = "(\w+) (\w+) bags contain no other ([bag?s?,?.]+)"
regex_input = "(\w+) (\w+) bags contain|(\d+) (\w+) (\w+) ([bag?s?,?.]+)|(no other bags.)"

def test_reg(text:str):
    for s in text.splitlines():
        yield re.findall(regex_input,s)

list(test_reg(test))


In [ ]:
regex_input = """\
(?P<agg>\w+) (?P<col>\w+) bags contain|\
(?P<num>\d+) (?P<cagg>\w+) (?P<ccol>\w+) ([bag?s?,?.]+)|\
(?P<no_bags>no other bags.)"""

def parse(iter_in):
    for s in iter_in :
        parsed_list = [mo.groupdict() for mo in re.finditer(regex_input, s)]
        k = "{agg}_{col}".format(**parsed_list[0])
        if len(parsed_list) == 2 and parsed_list[1]['no_bags']:
            yield { k: None }
        else :
            yield { k: {"{cagg}_{ccol}".format(**d): d['num'] for d in parsed_list[1:]} }

d = dict()
for dd in parse(test.splitlines()):
    d.update(dd)
d

In [ ]:
def part_1(iter_in):
    bags = dict()
    for d in parse(iter_in):
        bags.update(d)

    shiny_gold = set()
    search_stack = ["shiny_gold"]
    while len(search_stack):
        what = search_stack.pop()
        for k,v in bags.items():
            if v and what in v.keys():
                shiny_gold.add(k)
                search_stack.append(k)
    
    return shiny_gold

len(part_1(open("input/input7.txt")))
#part_1(test.splitlines())

In [ ]:
test = """\
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags."""

d = dict()
for dd in parse(test.splitlines()):
    d.update(dd)
d

In [ ]:
from anytree import Node

def part_2(iter_in):
    bags = dict()
    for d in parse(iter_in):
        bags.update(d)

    shiny_gold = Node("shiny_gold", num=1)
    search_stack = [shiny_gold]
    while len(search_stack):
        curr_node = search_stack.pop()
        if bags[curr_node.name] :
            for name,count in bags[curr_node.name].items():
                search_stack.append(Node(name, num=int(count), parent=curr_node))

    return shiny_gold


In [ ]:
from anytree import RenderTree, AsciiStyle, PreOrderIter

root = part_2(test.splitlines())
print(RenderTree(root,style=AsciiStyle()).by_attr())
[node.num for node in PreOrderIter(root)]

In [ ]:
from anytree import RenderTree, AsciiStyle, PreOrderIter

root = part_2(open("input/input7.txt"))
print(RenderTree(root,style=AsciiStyle()).by_attr())
[node.num for node in PreOrderIter(root)]

In [ ]:
bags["shiny_gold"]